# Accenture Scraper

Accenture (IT consulting - verdens største konsulenthus) https://www.accenture.com/us-en

This scraper collects texts on "Artificial Intelligence" from Accenture: https://www.accenture.com/us-en

## Libraries

Loading libraries...

In [1]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import re

import time

## URL List
The scraper collects urls from the returned search results on "artificial intelligence" filtered by content type "Insights": https://www.accenture.com/us-en/search/results?srk=Artificial%20Intelligence&pg=1&sb=1&filter=Insights

In [9]:
start_url = 'https://www.accenture.com/us-en/search/results?srk=artificial%20intelligence&pg=1&sb=1&filter=Insights'

In [82]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('button#onetrust-accept-btn-handler').click()
time.sleep(2)

pageSel = Selector(text = driver.page_source)

urldict = {}
urldict['titles'] = pageSel.css('h3.header.title.quarternary-title > a.search-result-link.cta::text').extract()
urldict['urls'] = pageSel.css('h3.header.title.quarternary-title > a.search-result-link.cta::attr(href)').extract()

y_coord = 4000
coordinates = {'x': 0, 'y': y_coord}
driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
time.sleep(3)

while True:
    try:
        driver.find_element_by_css_selector("div.reinvent-pagination-next-container > a.next-page-btn").click()
        time.sleep(3)
        driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
        time.sleep(3)
        
        pageSel = Selector(text = driver.page_source)
        
        titles = pageSel.css('h3.header.title.quarternary-title > a.search-result-link.cta::text').extract()
        urls = pageSel.css('h3.header.title.quarternary-title > a.search-result-link.cta::attr(href)').extract()
        
        urldict['titles'] = urldict['titles']+ titles
        urldict['urls'] = urldict['urls'] + urls
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No next button found")
        break

driver.quit()

No next button found


In [97]:
print(len(urldict['titles']),
     len(urldict['urls']))

520 519


In [84]:
index = 361
print(urldict['titles'][index], '\n',
      urldict['urls'][index])

Putting the Consumer Back into Consumer-packaged Goods | Accenture 
 /us-en/insights/strategy/mergers-acquisitions-technology


In [86]:
newdict = {}
newdict['urls'] = list(urldict['urls'])
newdict['titles'] = list(urldict['titles'])

for i in range(361,519):
    newdict['titles'][i] = list(urldict['titles'])[i+1]

Technology in Mergers & Acquisitions | Accenture  
 /us-en/insights/strategy/mergers-acquisitions-technology


In [107]:
newdict['titles'] = list(newdict['titles'])[0:519]
print(len(newdict['titles']),
     len(newdict['urls']))

519 519


In [108]:
index = 361
print(newdict['titles'][index], '\n',
      newdict['urls'][index])

Technology in Mergers & Acquisitions | Accenture  
 /us-en/insights/strategy/mergers-acquisitions-technology


In [112]:
urllist = list(map(lambda url: 'https://www.accenture.com/' + url, urldict['urls']))

### Save URLs

In [114]:
#URL dump
with open('../data_raw/urls/accenture_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

### Load URLs

In [115]:
# Load URLs
urllist = []
with open('../data_raw/urls/accenture_urls.txt', 'r') as f:
    for line in f:
        urllist.append(line.strip())

print(len(urllist),
      '\n',
     urllist[0])

519 
 https://www.accenture.com//us-en/insights/health/why-consumer-digital-health-adoption-stalling


## Parsing single page

In [116]:
urllist[0:10]

['https://www.accenture.com//us-en/insights/health/why-consumer-digital-health-adoption-stalling',
 'https://www.accenture.com//us-en/insights/health/unlocking-value-nurse-future',
 'https://www.accenture.com//us-en/insights/strategy/ai-potential',
 'https://www.accenture.com//us-en/insights/us-federal-government/leaders-in-action',
 'https://www.accenture.com//us-en/insights/us-federal-government/future-systems-create-more-value',
 'https://www.accenture.com//us-en/insights/digital/flexible-data-better-decisions',
 'https://www.accenture.com//us-en/insights/digital/virtual-workforce',
 'https://www.accenture.com//us-en/insights/high-tech/semiconductor-ecosystem',
 'https://www.accenture.com//us-en/insights/financial-services/technology-advisory-dark-data',
 'https://www.accenture.com//us-en/insights/us-federal-government/mission-value-with-ai']

In [217]:
page_url = urllist[2]
page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)
page_url

'https://www.accenture.com//us-en/insights/strategy/ai-potential'

In [218]:
page_dict = dict()
main_dict = dict()

In [219]:
date_regex = r'(?<=\"issueDate\":\")\d{4}\-\d{1,2}\-\d{1,2}'
type_regex = r'(?<=\"contentType\":\")\w{2,15}(?=\",)'

page_scripts = page_sel.css('script ::text').extract()
page_scripts = ' '.join(page_scripts)

siteno = 1

page_id = 'site' + str(siteno)

page_title = page_sel.css('title ::text').extract_first()
page_date = re.findall(date_regex, page_scripts)[0]
page_dldate = date.today().strftime('%Y/%m/%d')
page_type = re.findall(type_regex, page_scripts)[0]
page_text = page_sel.css('div.block-content > div > div > div > article ::text').extract()
page_text.append(' '.join(page_sel.css('div#content-wrapper > section > div > div > section > div > div > div.component.richtext ::text').extract()))
page_text = ' '.join(page_text)

page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['download date'] = page_dldate
page_dict['text type'] = page_type
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [221]:
print(len(main_dict),
      main_dict['site1'])

1 {'url': 'https://www.accenture.com//us-en/insights/strategy/ai-potential', 'title': 'Artificial Intelligence (AI) Potential | Accenture', 'article date': '2020-03-04', 'download date': '2020/03/10', 'text type': 'insights', 'text': "\r\n             \r\n                     RESEARCH REPORT \r\n                                     In brief \r\n                     In brief \r\n                                     \r\n                                 What do industries as diverse as Energy, Industrial Equipment and Automotive have in common? They are all capital intensive and struggle to scale AI. \r\n                                 Many companies achieve value in pockets. To realize AI's exponential value, leaders need to outline a clear strategy based on an integrated picture. \r\n                                 Leaders will prioritize AI investments based on relative value, complexity, time to market and potential synergies across company areas. \r\n                               

## Accenture Spider

In [223]:
from scrapy.crawler import CrawlerProcess

# Create class
class accenture_spider (scrapy.Spider):
    name = "accenture_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
                
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        date_regex = r'(?<=\"issueDate\":\")\d{4}\-\d{1,2}\-\d{1,2}'
        type_regex = r'(?<=\"contentType\":\")\w{2,15}(?=\",)'

        page_scripts = page_sel.css('script ::text').extract()
        page_scripts = ' '.join(page_scripts)

        page_id = 'site' + str(siteno)

        page_title = page_sel.css('title ::text').extract_first()
        page_date = re.findall(date_regex, page_scripts)[0]
        page_dldate = date.today().strftime('%Y/%m/%d')
        page_type = re.findall(type_regex, page_scripts)[0]
        page_text = page_sel.css('div.block-content > div > div > div > article ::text').extract()
        page_text.append(' '.join(page_sel.css('div#content-wrapper > section > div > div > section > div > div > div.component.richtext ::text').extract()))
        page_text = ' '.join(page_text)

        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['download date'] = page_dldate
        page_dict['text type'] = page_type
        page_dict['text'] = page_text
        page_dict['agency'] = "Accenture"

        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(accenture_spider)
process.start()

2020-03-10 14:27:35 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2020-03-10 14:27:35 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.18362-SP0
2020-03-10 14:27:35 [scrapy.crawler] INFO: Overridden settings: {}
2020-03-10 14:27:35 [scrapy.extensions.telnet] INFO: Telnet Password: 5c0219c061fdb6d4
2020-03-10 14:27:35 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-03-10 14:27:36 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

https://www.accenture.com//us-en/insights/digital/flexible-data-better-decisions


2020-03-10 14:27:38 [urllib3.connectionpool] DEBUG: https://www.accenture.com:443 "GET //us-en/insights/digital/flexible-data-better-decisions HTTP/1.1" 200 162526
2020-03-10 14:27:38 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.accenture.com:443


https://www.accenture.com//us-en/insights/strategy/ai-potential


2020-03-10 14:27:39 [urllib3.connectionpool] DEBUG: https://www.accenture.com:443 "GET //us-en/insights/strategy/ai-potential HTTP/1.1" 200 159717
2020-03-10 14:27:39 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.accenture.com:443


...

2020-03-10 14:36:49 [urllib3.connectionpool] DEBUG: https://www.accenture.com:443 "GET //us-en/insight-global-risk-study-2017 HTTP/1.1" 200 226829
2020-03-10 14:36:49 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.accenture.com:443


https://www.accenture.com//us-en/insight-2016-upstream-oil-gas-digital-trends-survey


2020-03-10 14:36:50 [urllib3.connectionpool] DEBUG: https://www.accenture.com:443 "GET //us-en/insight-2016-upstream-oil-gas-digital-trends-survey HTTP/1.1" 200 194757
2020-03-10 14:36:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.accenture.com/us-en/services/industryx0-index> (referer: None)
2020-03-10 14:36:50 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.accenture.com:443


https://www.accenture.com/us-en/services/industryx0-index


2020-03-10 14:36:51 [urllib3.connectionpool] DEBUG: https://www.accenture.com:443 "GET /us-en/services/industryx0-index HTTP/1.1" 200 174908
2020-03-10 14:36:51 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.accenture.com/us-en/services/industryx0-index> (referer: None)
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\twisted\internet\defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-223-16b4bf30b817>", line 36, in parse
    page_type = re.findall(type_regex, page_scripts)[0]
IndexError: list index out of range
2020-03-10 14:36:51 [scrapy.core.engine] INFO: Closing spider (finished)
2020-03-10 14:36:51 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 150700,
 'downloader/request_count': 512,
 'downloader/request_method_count/GET': 512,
 'downloader/response_bytes': 78746469,
 'downloader/response_count': 512,
 'downloader/res

In [224]:
process.stop()

<DeferredList at 0x1ea49e5d2b0 current result: []>

In [225]:
len(main_dict)

507

In [226]:
import json

with open('..\\data_raw\\articles\\accenture_articles.json', 'w') as file:
    json.dump(main_dict, file)